In [2]:
!pip install torch torchtext
!git clone https://github.com/neubig/nn4nlp-code.git

    100% |████████████████████████████████| 484.0MB 24kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5c906000 @  0x7ffaad8f71c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 51kB 11.0MB/s 
    100% |████████████████████████████████| 51kB 7.0MB/s 
  Running setup.py bdist_wheel for torchtext ... - \ done
  Stored in directory: /content/.cache/pip/wheels/42/a6/f4/b267328bde6bb680094a0c173e8e5627ccc99543abded97204
Successfully built torchtext
Cloning into 'nn4nlp-code'...
remote: Counting objects: 372, done.


remote: Total 372 (delta 0), reused 0 (delta 0), pack-reused 372
Receiving objects: 100% (372/372), 6.33 MiB | 22.04 MiB/s, done.
Resolving deltas: 100% (131/131), done.


In [0]:
from __future__ import print_function
import time

from collections import defaultdict
import random
import math
import sys
import argparse

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

In [0]:
# format of files: each line is "word1|tag1 word2|tag2 ..."
train_file = "nn4nlp-code/data/tags/train.txt"
dev_file = "nn4nlp-code/data/tags/dev.txt"

w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))


def read(fname):
    """
    Read tagged file
    """
    with open(fname, "r") as f:
        for line in f:
            words, tags = [], []
            for wt in line.strip().split():
                w, t = wt.split('|')
                words.append(w2i[w])
                tags.append(t2i[t])
            yield (words, tags)


# Read the data
train = list(read(train_file))
unk_word = w2i["<unk>"]
w2i = defaultdict(lambda: unk_word, w2i)
unk_tag = t2i["<unk>"]
t2i = defaultdict(lambda: unk_tag, t2i)
nwords = len(w2i)
ntags = len(t2i)
dev = list(read(dev_file))

In [45]:
t2i

defaultdict(<function __main__.<lambda>>,
            {'<unk>': 9,
             'B-LOC': 7,
             'B-MISC': 5,
             'B-ORG': 8,
             'B-PER': 6,
             'I-LOC': 4,
             'I-MISC': 0,
             'I-ORG': 3,
             'I-PER': 2,
             'O': 1})

In [5]:
!head nn4nlp-code/data/tags/train.txt

The|I-MISC Oxford|I-MISC Companion|I-MISC to|I-MISC Philosophy|I-MISC says|O ,|O "|O there|O is|O no|O single|O defining|O position|O that|O all|O anarchists|O hold|O ,|O and|O those|O considered|O anarchists|O at|O best|O share|O a|O certain|O family|O resemblance|O .|O "|O
In|O the|O end|O ,|O for|O anarchist|O historian|O Daniel|I-PER Guerin|I-PER "|O Some|O anarchists|O are|O more|O individualistic|O than|O social|O ,|O some|O more|O social|O than|O individualistic|O .|O
From|O this|O climate|O William|I-PER Godwin|I-PER developed|O what|O many|O consider|O the|O first|O expression|O of|O modern|O anarchist|O thought|O .|O
Godwin|I-PER was|O ,|O according|O to|O Peter|I-PER Kropotkin|I-PER ,|O "|O the|O first|O to|O formulate|O the|O political|O and|O economical|O conceptions|O of|O anarchism|O ,|O even|O though|O he|O did|O not|O give|O that|O name|O to|O the|O ideas|O developed|O in|O his|O work|O "|O ,|O while|O Godwin|I-PER attached|O his|O anarchist|O ideas|O to|O an|O early|O

In [0]:
class BiLSTM(nn.Module):
  def __init__(self, vocab_size, tag_size, embed_dim, hidden_size, num_layers=1):
    super(BiLSTM, self).__init__()
    
    self.vocab_size = vocab_size
    self.tag_size = tag_size
    self.embed_dim = embed_dim
    self.hidden_size = hidden_size
    
    self.embeddings = nn.Embedding(vocab_size, embed_dim)
    self.lstm = nn.LSTM(embed_dim, hidden_size, num_layers, bidirectional=True)
    self.fc = nn.Linear(hidden_size*2, tag_size)
  
  def init_hidden(self, bs):
    return (torch.zeros(2, bs, self.hidden_size, device=device), 
            torch.zeros(2, bs, self.hidden_size, device=device))
  
  def forward(self, x):
    seq_len, bs = x.shape
    h = self.init_hidden(bs)
    x = self.embeddings(x)
    x, h = self.lstm(x, h)
    x = self.fc(x.view(seq_len*bs, -1)).view(seq_len, bs, -1)
    return x

In [0]:
class TaggerDataset(Dataset):
  def __init__(self, data):
    self.data = data
  
  def __len__(self):
    return len(self.data)
  
  def __getitem__(self, ix):
    return torch.LongTensor(self.data[ix][0]), torch.LongTensor(self.data[ix][1])

In [0]:
def pad_tensor(vec, pad, dim):
    """
    args:
        vec - tensor to pad
        pad - the size to pad to
        dim - dimension to pad

    return:
        a new tensor padded to 'pad' in dimension 'dim'
    """
    pad_size = list(vec.shape)
    pad_size[dim] = pad - vec.size(dim)
    return torch.cat([vec, torch.zeros(*pad_size).long()], dim=dim)


class PadCollate:
    """
    a variant of callate_fn that pads according to the longest sequence in
    a batch of sequences
    """

    def __init__(self, dim=0):
        """
        args:
            dim - the dimension to be padded (dimension of time in sequences)
        """
        self.dim = dim

    def pad_collate(self, batch):
        """
        args:
            batch - list of (tensor, label)

        reutrn:
            xs - a tensor of all examples in 'batch' after padding
            ys - a LongTensor of all labels in batch
        """
        # find longest sequence
        xlens = list(map(lambda x: x[0].shape[self.dim], batch))
        ylens = list(map(lambda x: x[1].shape[self.dim], batch))
        max_xlen = max(xlens)
        max_ylen = max(ylens)
        # pad according to max_len
        batch = list(map(lambda x:
                    (pad_tensor(x[0], pad=max_xlen, dim=self.dim), 
                    pad_tensor(x[1], pad=max_ylen, dim=self.dim)), batch))
        # stack all
        xs = torch.stack(list(map(lambda x: x[0], batch)), dim=0)
        ys = torch.stack(list(map(lambda x: x[1], batch)), dim=0)
        return xs, ys, torch.LongTensor(xlens), torch.LongTensor(ylens)

    def __call__(self, batch):
        return self.pad_collate(batch)


In [0]:
embed_dim = 128
hidden_dim = 64
BATCH_SIZE = 16

model = BiLSTM(nwords, ntags, embed_dim, hidden_dim)

criterion = nn.CrossEntropyLoss(ignore_index=9)
trainer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [0]:
train_dataset = TaggerDataset(train)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=PadCollate(dim=0))

dev_dataset = TaggerDataset(dev)
dev_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=PadCollate(dim=0))

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [55]:
for epoch_i in range(10):
  model.train()
  total_loss = 0.
  for batch_i, batch in enumerate(train_dataloader):
    xs, ys, xlens, ylens = batch
    logits = model(xs.transpose(0, 1))

    model.zero_grad()
    loss = criterion(logits.transpose(0,1).contiguous().view(-1, ntags), ys.view(-1))
    total_loss += loss.item()
    loss.backward()

    trainer.step()

    if batch_i % 200 == 0:
      print(f"epoch {epoch_i} | batch {batch_i} | avg_loss: {total_loss/(batch_i+1):.4f}")
    
  model.eval()
  val_loss = 0.
  val_correct = 0
  val_cnt = 0
  for batch_i, batch in enumerate(dev_dataloader):
    xs, ys, xlens, ylens = batch
    logits = model(xs.transpose(0, 1))
    loss = criterion(logits.transpose(0,1).contiguous().view(-1, ntags), ys.view(-1))
    val_loss += loss.item()
    mask = torch.arange(ys.shape[1]).unsqueeze(0).expand_as(ys) - ylens.float().unsqueeze(1) < 0 
    correct = ((torch.argmax(logits, -1).transpose(0, 1) == ys)  & mask).sum()
    val_cnt += ylens.sum().item()
    val_correct += correct.item()
  print(f"val_acc: {1. * val_correct / val_cnt} | val_loss: {val_loss/(batch_i+1)}")

epoch 0 | batch 0 | avg_loss: 0.0035
epoch 0 | batch 200 | avg_loss: 0.0027
epoch 0 | batch 400 | avg_loss: 0.0027
epoch 0 | batch 600 | avg_loss: 0.0027
val_acc: 0.9639813254420694 | val_loss: 0.0020012864457676187
epoch 1 | batch 0 | avg_loss: 0.0013
epoch 1 | batch 200 | avg_loss: 0.0017
epoch 1 | batch 400 | avg_loss: 0.0019
epoch 1 | batch 600 | avg_loss: 0.0018
val_acc: 0.9640700215289593 | val_loss: 0.0013139211153116775
epoch 2 | batch 0 | avg_loss: 0.0049
epoch 2 | batch 200 | avg_loss: 0.0011
epoch 2 | batch 400 | avg_loss: 0.0011
epoch 2 | batch 600 | avg_loss: 0.0012
val_acc: 0.9641506543352228 | val_loss: 0.0009995323370341795
epoch 3 | batch 0 | avg_loss: 0.0003
epoch 3 | batch 200 | avg_loss: 0.0009
epoch 3 | batch 400 | avg_loss: 0.0008
epoch 3 | batch 600 | avg_loss: 0.0008
val_acc: 0.9642716035446182 | val_loss: 0.0004485483444899728
epoch 4 | batch 0 | avg_loss: 0.0004
epoch 4 | batch 200 | avg_loss: 0.0005
epoch 4 | batch 400 | avg_loss: 0.0004
epoch 4 | batch 600 |